In [ ]:
import os
import json
import sys
import sqlite3

RAW_APK_FOLDER = "./apks.json" # This is the folder where the raw APKs (downloaded from the Play Store) are stored
DB = "" # This is the path to the SQLite database that contains the information about the apps

def read_db_package_names(db_file):
    """
        Reads the SQLite database and selects all apps that have at least 1M installs
    """
    conn = sqlite3.connect(db_file)
    c = conn.cursor()
    c.execute("SELECT appId FROM app WHERE minInstall >= 1000000")
    results = c.fetchall()
    conn.close()
    results = [i[0] for i in results]
    return results

    
    
all_files = os.listdir(RAW_APK_FOLDER)
amount_all_files = len(all_files)
package_names = read_db_package_names(DB)
print("Found " + str(len(package_names)) + " packages in the database.")

split_apks = []
single_apk = []
no_apk = []

for i in range(0, len(package_names)):
        
    progress = i / len(package_names)
    bar_length = 20
    block = int(round(bar_length * progress))
    text = "\rProgress: [{0}] {1:.1f}%".format("#" * block + "-" * (bar_length - block), progress * 100)
    sys.stdout.write(text)
    sys.stdout.flush()

    package_name = package_names[i]

    results = [i for i in all_files if (i.startswith(package_name + ".split.") and i.endswith(".zip")) or i == package_name + ".apk"]
    if (len(results) > 1):
        split_apks.append((package_name, len(results)))
    elif len(results) == 1:
        single_apk.append(package_name)
    else:
        no_apk.append(package_name)